In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, DepthwiseConv2D

Question 1 -
Implement 3 different CNN architectures with a comparison table for the MNSIT
dataset using the Tensorflow library.
Note -
1. The model parameters for each architecture should not be more than 8000
parameters
2. Code comments should be given for proper code understanding.
3. The minimum accuracy for each accuracy should be at least 96%

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
x_train.shape

(60000, 28, 28)

In [ ]:
# Normalise the pixel values to range 0-1 and reshape
x_train = x_train.reshape((x_train.shape[0], 28,28,1)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0],28,28,1)).astype('float32') / 255

In [ ]:
#one hot encoding the lables
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess the dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize and reshape the data
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)).astype('float32') / 255

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Architecture 1: Small CNN with 6,810 parameters
model1 = models.Sequential([
    layers.Conv2D(4, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])

# Architecture 2: Depthwise CNN with 6,650  parameters
model2 = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.DepthwiseConv2D((3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])

# Architecture 3: Strided CNN with 5,930 parameters
model3 = models.Sequential([
    layers.Conv2D(20, (3, 3), strides=(2,2), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])


# Function to compile, train, and evaluate models
def train_and_evaluate(model, x_train, y_train, x_test, y_test, optimizer='adam', epochs=5, batch_size=64):
    print("optimzer", optimizer, "epochs", epochs, "batch_size", batch_size)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    return test_acc, model.count_params()

# Train and evaluate all models
results = {}
models_list = [model1, model2, model3]
model_names = ['Model 1', 'Model 2', 'Model 3']
optimizers = ['adam', 'sgd', 'rmsprop']
epochs = [15, 10, 7]

for model, name, optimizer, epoch in zip(models_list, model_names, optimizers, epochs):
    acc, params = train_and_evaluate(model, x_train, y_train, x_test, y_test, optimizer, epoch)
    results[name] = {'Accuracy': acc, 'Parameters': params, 'Optimizer':optimizer}

# Print results in a comparison table
print(f"{'Model':<10} {'Accuracy':<10} {'Parameters':<10} {'Optimizer':<10}")
for model, metrics in results.items():
    print(f"{model:<10} {metrics['Accuracy']:<10.4f} {metrics['Parameters']:<10} {metrics['Optimizer']:10}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


optimzer adam epochs 15 batch_size 64
optimzer sgd epochs 10 batch_size 64
optimzer rmsprop epochs 7 batch_size 64
Model      Accuracy   Parameters Optimizer 
Model 1    0.9760     6810       adam      
Model 2    0.9748     6650       sgd       
Model 3    0.9718     7410       rmsprop   


Question 2 -
Implement 5 different CNN architectures with a comparison table for CIFAR 10
dataset using the PyTorch library
Note -
1. The model parameters for each architecture should not be more than 10000
parameters
2. Code comments should be given for proper code understanding

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary

# Define transformations for the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

# Define the CNN architectures

# Model 1: Simple CNN
def model_1():
    return nn.Sequential(
        nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1),  # 8 filters
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),                # 16x16x8
        nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1), # 16 filters
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),                # 8x8x16
        nn.Flatten(),
        nn.Linear(16 * 8 * 8, 10)                             # Fully connected layer
    )

# Model 2: Deeper CNN
def model_2():
    return nn.Sequential(
        nn.Conv2d(3, 6, kernel_size=3, stride=1, padding=1),  # 6 filters
        nn.ReLU(),
        nn.Conv2d(6, 12, kernel_size=3, stride=1, padding=1), # 12 filters
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),                # 16x16x12
        nn.Conv2d(12, 24, kernel_size=3, stride=1, padding=1),# 24 filters
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),                # 8x8x24
        nn.Flatten(),
        nn.Linear(24 * 8 * 8, 10)                             # Fully connected layer
    )

# Model 3: Compact CNN
def model_3():
    return nn.Sequential(
        nn.Conv2d(3, 4, kernel_size=3, stride=1, padding=1),  # 4 filters
        nn.ReLU(),
        nn.Conv2d(4, 8, kernel_size=3, stride=1, padding=1),  # 8 filters
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),                # 16x16x8
        nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1), # 16 filters
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),                # 8x8x16
        nn.Flatten(),
        nn.Linear(16 * 8 * 8, 10)                             # Fully connected layer
    )

# Training and evaluation function
def train_and_evaluate(model, trainloader, testloader, epochs=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader):.4f}")

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy: {100 * correct / total:.2f}%")

# Instantiate and summarize models
for i, model_fn in enumerate([model_1], 1):
    print(f"\nModel {i} Summary:")
    model = model_fn()
    print("model--->", model)
    # summary(model, (3, 32, 32))
    train_and_evaluate(model, trainloader, testloader)

# Comparison Table:
# | Model   | Parameters | Test Accuracy |
# |---------|------------|---------------|
# | Model 1 | ~7,000     | TBD           |
# | Model 2 | ~9,000     | TBD           |
# | Model 3 | ~8,500     | TBD           |


Files already downloaded and verified
Files already downloaded and verified

Model 1 Summary:
model---> Sequential(
  (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=1024, out_features=10, bias=True)
)
Epoch 1, Loss: 1.5099
Epoch 2, Loss: 1.2314
Epoch 3, Loss: 1.1173
Epoch 4, Loss: 1.0482
Epoch 5, Loss: 1.0059
Accuracy: 62.43%

Model 2 Summary:
model---> Sequential(
  (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(12, 24, kerne